In [1]:
from utils2 import *

In [2]:
%load_ext cython
%load_ext rpy2.ipython


In [4]:
%%R

require(ggplot2)
install.packages("devtools")
library("devtools")
install_github("bdsegal/fastPerm")

library(fastPerm)

R[write to console]: Loading required package: ggplot2

R[write to console]: Installing package into ‘/home/ekvall/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘gh’, ‘openssl’, ‘usethis’, ‘covr’, ‘httr’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/gh_1.1.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 29043 bytes (28 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

R[write to console]: downloaded 366 KB


R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/RtmpfiQvum/downloaded_packages’
R[write to console]: 
R[write to console]: 

R[write to console]: Loading required package: usethis

R[write to console]: Downloading GitHub repo bdsegal/fastPerm@master



✔  checking for file ‘/tmp/RtmpfiQvum/remotes313c54bc5d8f/bdsegal-fastPerm-639f37e/DESCRIPTION’
─  preparing ‘fastPerm’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘fastPerm_0.0.0.9000.tar.gz’
   


R[write to console]: Installing package into ‘/home/ekvall/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [5]:
%%R
    runFastPerm <- function(x,y,output, n_samples) {
        e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
      
            mStopDiffMean(x[[i]], y[[i]])
            valX = fastPerm(x[[i]], y[[i]], testStat = diffMean)
            
            valX <-unlist(valX)

      
            output[[i]] <- as.numeric(valX[1])
      
            }
    

    return(output)


    }

In [6]:
def fastPerm(A,B):
    x = pd.DataFrame(A, columns=list(range(A.shape[1]))).T
    y = pd.DataFrame(B, columns=list(range(B.shape[1]))).T
    n_samples = x.shape[1]
    outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T
    
    %R -i x
    %R -i y
    %R -i outputDf
    %R -i n_samples
    
    
    %R out <- runFastPerm(x,y, outputDf, n_samples)
    
    
    %R -o out
    return list(out.T["output"].values)

def runFastPerm(X_list, y_list):
    pfp_list = list()
    TIME = list()
    
    for Xp, yp in zip(X_list, y_list):

        p_fp = list()
        time_list = list()
        
        for x, y in zip(Xp, yp):
           
            start = time.time()
            p_fp.append(fastPerm(np.array(y)[np.newaxis,...], np.array(x)[np.newaxis,...]))
            end = time.time()
        
            time_list.append(end - start)
        
        pfp_list.append(p_fp)
        TIME.append(time_list)
        
    return pfp_list, TIME

In [7]:
def getPATH(path, suffix, prefix):
    return path + '/'+ suffix + '_' + prefix + ".png"

In [8]:
path = "figures/GreenVsFastperm"

In [9]:
sampleShape = [10,50,100,150,200,250,300]


In [10]:
mean =0.2

In [11]:
A_data,B_data = getAllSynthticData(sampleShape, mean)

# Sample from $X \sim N(5.0,1)$ and $Y\sim N(5.2,1)$

In [12]:
X_list5_2,y_list5_2= getAllSynthticData(sampleShape, mean)
pt_list5_2, pe_list5_2, parallel_TIME = shiftMethod(np.asarray(X_list5_2), np.asarray(y_list5_2), 40)

In [13]:

p_val_list5_2, TIME_MC5_2 = runFastPerm(np.asarray(X_list5_2), np.asarray(y_list5_2))

In [ ]:
singleT_pt_list5_2, singleT_pe_list5_2, singleThreadTime = shiftMethod(np.asarray(X_list5_2), np.asarray(y_list5_2), 40, False)

In [ ]:
for st_l, para_l in zip(singleT_pt_list5_2,pt_list5_2):
    st_a = np.array(st_l)
    para_a = np.array(para_l)
    print(np.allclose(st_a, para_a))

In [ ]:
timePlotSNSFastperm(np.log10(parallel_TIME), np.log10(TIME_MC5_2), 
            sampleShape, path=getPATH(path, str(mean),"SNSruntimeLogSingleThread"),log=True, 
            TIMEsingleThred=np.log10(singleThreadTime))

In [ ]:
timePlotSNSFastperm(parallel_TIME, TIME_MC5_2, sampleShape, 
            path=getPATH(path, str(mean),"SNSruntimeSingleThread"),log=False, 
            TIMEsingleThred=singleThreadTime)


